# Utilities

This notebook contains utility functions that will be loaded in other notebooks.

In [2]:
import os
import numpy as np
import pandas as pd
from tqdm.notebook import trange, tqdm
from IPython.display import display, HTML
from copy import deepcopy
import random
import pickle

from tti_explorer import config, utils
from tti_explorer.case import simulate_case, CaseFactors
from tti_explorer.contacts import EmpiricalContactsSimulator
from tti_explorer.strategies import TTIFlowModel, RETURN_KEYS

import warnings
warnings.filterwarnings('ignore')

## Utility Functions

### Sensitivity Analysis
Contains utility functions related to sensitivity analysis experiments.

In [ ]:
def print_doc(func):
    """
    Used to show description of a function.
    
    :param func: function object to be described.
    """
    
    print(func.__doc__)

In [ ]:
def load_csv(pth):
    """
    Used to load csv data.
    
    :param pth: path to file.
    
    :return: numpy loaded file.
    """
    
    return np.loadtxt(pth, dtype=int, skiprows=1, delimiter=",")

In [ ]:
def do_simulation(policy_name, 
                  case_config, 
                  contacts_config, 
                  n_cases, 
                  random_state=None, 
                  dict_params=None):
    """
    Conduct simulation.
    
    :param policy_name: the policy name (e.g., S1_no_TTI, S2_symptom_based_TTI, etc)
    :param case_config: case configuration
    :param contacts_config: contacts configuration
    :param n_cases: number of primary cases to generate.
    :param random_state: seed
    :param dict_params: parameter and value for the policy config.
    
    :return: outputs of the simulation.
    """
    
    ## Prepare simulation configuration
    case_config = case_config
    contacts_config = contacts_config
    
    if dict_params is None:
        policy_config = config.get_strategy_configs("delve", policy_name)[policy_name]
        factor_config = utils.get_sub_dictionary(policy_config, config.DELVE_CASE_FACTOR_KEYS)
        strategy_config = utils.get_sub_dictionary(policy_config, config.DELVE_STRATEGY_FACTOR_KEYS)
    else:
        policy_config = config.get_strategy_configs("delve", policy_name)[policy_name]
        for key, value in dict_params.items():
            policy_config[key] = value
            factor_config = utils.get_sub_dictionary(policy_config, config.DELVE_CASE_FACTOR_KEYS)
            strategy_config = utils.get_sub_dictionary(policy_config, config.DELVE_STRATEGY_FACTOR_KEYS)

    ## Initialise configuration model
    if random_state is None:
        rng = np.random.RandomState(random.randint(0, 1000))
    else:
        rng = np.random.RandomState(random_state)

    simulate_contacts = EmpiricalContactsSimulator(over18, under18, rng)
    tti_model = TTIFlowModel(rng, **strategy_config)

    # Aggregates all cases outputs
    outputs = list()

    ## Perform simulation
    for _ in range(n_cases):
        case = simulate_case(rng, **case_config)
        case_factors = CaseFactors.simulate_from(rng, case, **factor_config)
        contacts = simulate_contacts(case, **contacts_config)
        res = tti_model(case, contacts, case_factors)
        outputs.append(res)
    
    return outputs

In [ ]:
def summarise_simulation_results(outputs):
    """
    Summarise simulation results in a table.
    
    :param outputs: outputs of the simulation
    """
    
    to_show = [
        RETURN_KEYS.base_r,
        RETURN_KEYS.reduced_r,
        RETURN_KEYS.man_trace,
        RETURN_KEYS.app_trace,
        RETURN_KEYS.tests
    ]

    # Scale factor to turn simulation numbers into UK population numbers
    nppl = 120
    scales = [1, 1, nppl, nppl, nppl]

    results = pd.DataFrame(
        outputs
    ).mean(
        0
    ).loc[
        to_show
    ].mul(
        scales
    ).to_frame(
        name=f"Simulation results: "
    ).rename(
        index=lambda x: x + " (k per day)" if x.startswith("#") else x
    )

    display(results.round(1))

In [ ]:
def compile_df_result(dict_result, str_param, params, n_experiments):
    """
    Used to compile df_result from simulation.
    
    :param dict_result: dictionary filled by simulation outputs
    :param str_param: the string identifier of param
    :param params: parameters value
    :param n_experiments: number of experiments
    
    :return df_result: dataframe aggregating result
    """
    
    key_params = [f"{i}_{j}" for i in ["S1", "S2", "S3", "S4", "S5"] for j in ["no_TTI", "symptom_based_TTI", "test_based_TTI", "test_based_TTI_test_contacts"]]
    df_result = {"strategy": [], 
                 "gov_policy": [],
                 str_param: [],
                 "effective_r": [], 
                 "base_r": [],
                 "manual_traces": [], 
                 "app_traces": [], 
                 "test_needed": [],
                 "persondays_quarantined": [],
                 "secondary_infections": [],
                 "secondary_infections_social": [],
                 "secondary_infections_isolating": [],
                 "secondary_infections_contact": []}
    
    nppl = 120 # default value for case config in thousands

    for key in key_params:
        strategy = key[:2]
        gov_policy = key[3:]

        for param in params:
            for i in range(n_experiments):
                result_format = pd.DataFrame(dict_result[key][param][i]).mean(0)
                df_result[str_param].append(param)
                df_result["strategy"].append(strategy)
                df_result["gov_policy"].append(gov_policy)
                df_result["effective_r"].append(result_format.loc["Effective R"])
                df_result["base_r"].append(result_format.loc["Base R"])
                df_result["manual_traces"].append(result_format.loc["# Manual Traces"] * nppl)
                df_result["app_traces"].append(result_format.loc["# App Traces"] * nppl)
                df_result["test_needed"].append(result_format.loc["# Tests Needed"] * nppl)
                df_result["persondays_quarantined"].append(result_format.loc["# PersonDays Quarantined"] * nppl)
                df_result["secondary_infections"].append(result_format.loc["# Secondary Infections"] * nppl)
                df_result["secondary_infections_social"].append(result_format.loc["# Secondary Infections Prevented by Social Distancing"] * nppl)
                df_result["secondary_infections_isolating"].append(result_format.loc["# Secondary Infections Prevented by Isolating Cases with Symptoms"] * nppl)
                df_result["secondary_infections_contact"].append(result_format.loc["# Secondary Infections Prevented by Contact Tracing"] * nppl)
    
    df_result = pd.DataFrame(df_result)
    
    return df_result

In [ ]:
def plot_axis_variation(x, y, df_result, ci, xlabel, ylabel, xticks=None):
    """
    Used to plot axis variation result for non-gp sensitivity analysis.
    
    :param x: x column in dataframe
    :param y: y column in dataframe
    :param df_result: dataframe containing results
    :param ci: confidence interval level
    :param xlabel: x label
    :param ylabel: y label
    """
    
    fig = plt.figure(figsize=(12, 16), facecolor="white")
    ax1 = fig.add_subplot(321)
    ax2 = fig.add_subplot(322)
    ax3 = fig.add_subplot(323)
    ax4 = fig.add_subplot(324)
    ax5 = fig.add_subplot(325)

    # Line plot
    sns.lineplot(x=x, y=y, 
                 data=df_result[df_result["strategy"] == "S1"], 
                 hue="gov_policy", err_style="bars", linewidth=1,
                 ci=ci, ax=ax1, markers=True)
    sns.lineplot(x=x, y=y, 
                 data=df_result[df_result["strategy"] == "S2"], 
                 hue="gov_policy", err_style="bars", linewidth=1,
                 ci=ci, ax=ax2, markers=True)
    sns.lineplot(x=x, y=y, 
                 data=df_result[df_result["strategy"] == "S3"], 
                 hue="gov_policy", err_style="bars", linewidth=1,
                 ci=ci, ax=ax3, markers=True)
    sns.lineplot(x=x, y=y, 
                 data=df_result[df_result["strategy"] == "S4"], 
                 hue="gov_policy", err_style="bars", linewidth=1,
                 ci=ci, ax=ax4, markers=True)
    sns.lineplot(x=x, y=y, 
                 data=df_result[df_result["strategy"] == "S5"], 
                 hue="gov_policy", err_style="bars", linewidth=1,
                 ci=ci, ax=ax5, markers=True)

    # Display one legend
    handles, labels = ax1.get_legend_handles_labels()
    ax1.legend(handles=handles[1:], labels=["No TTI", "Symptoms Based TTI", "Test Based TTI", "Test Based TTI - Test Contacts"], loc='center left', 
               bbox_to_anchor=(0.07, 1.09), ncol=4, frameon=False,
              prop={'size': 12})

    # Remove legends
    for ax in [ax2, ax3, ax4, ax5]:
        legend = ax.legend()
        legend.remove()

    # Rename axis
    ax1.set_xlabel(f"{xlabel} (S1)", fontsize=13)
    ax1.set_ylabel(ylabel, fontsize=13)


    ax2.set_xlabel(f"{xlabel} (S2)", fontsize=13)
    ax2.set_ylabel(ylabel, fontsize=13)


    ax3.set_xlabel(f"{xlabel} (S3)", fontsize=13)
    ax3.set_ylabel(ylabel, fontsize=13)


    ax4.set_xlabel(f"{xlabel} (S4)", fontsize=13)
    ax4.set_ylabel(ylabel, fontsize=13)


    ax5.set_xlabel(f"{xlabel} (S5)", fontsize=13)
    ax5.set_ylabel(ylabel, fontsize=13)
    
    # Despine
    for ax in [ax1, ax2, ax3, ax4, ax5]:
        ax.spines['bottom'].set_color('#D3D3D3')
        ax.spines['left'].set_color('#D3D3D3')
        ax.spines['right'].set_color('#D3D3D3')
        ax.spines['top'].set_color('#D3D3D3')
        ax.tick_params(axis='both', which='major', labelsize=11)
        ax.set_facecolor("white")

        for axis in ['top','bottom','left','right']:
            ax.spines[axis].set_linewidth(0)

    plt.show()

In [1]:
def get_gp_config():
    """
    Default configuration for all GP experiments.
    """
    
    # Parameters
    p_under18 = 0.21 
    home_sar = 0.3 
    work_sar = 0.045 
    other_sar = 0.045
    period = 10 
    compliance = 0.8 
    strategy_quarantine_length = 14 
    testing_delay = 2 
    app_trace_delay = 0 
    manual_trace_delay = 1
    app_coverage = 0.35 
    go_to_school_prob = 0.5
    wfh_prob = 0.45 
    max_contacts = 10 
    
    # Case
    case_config = {
        'p_under18': p_under18, 
        'infection_proportions': {
            'dist': [0.8333333333333334, 0.1, 0.06666666666666667], 
            'nppl': 120
            }, 
        'p_day_noticed_symptoms': [0, 0.25, 0.25, 0.2, 0.1, 0.05, 0.05, 0.05, 0.05, 0.0], 
        'inf_profile': [0.046966101377360424, 0.15602255610429985, 0.19829974712514023, 0.18356485224565827, 0.14541407040442172, 0.10500447388376151, 0.07130993362939089, 0.04635772205156416, 0.029167894888682697, 0.017892648289720214]
    }
    
    # Contact
    contacts_config = {
       'home_sar': home_sar, 
       'work_sar': work_sar, 
       'other_sar': other_sar, 
       'period': period, 
       'asymp_factor': 0.5 # not useful
    }
    
    # Policy
    policy_config = {
        'isolate_individual_on_symptoms': True, 
        'isolate_individual_on_positive': True, 
        'isolate_household_on_symptoms': True, 
        'isolate_household_on_positive': True, 
        'isolate_contacts_on_symptoms': False, 
        'isolate_contacts_on_positive': True, 
        'test_contacts_on_positive': False, 
        'do_symptom_testing': True, 
        'do_manual_tracing': True, 
        'do_app_tracing': True, 
        'fractional_infections': True, 
        'testing_delay': testing_delay, 
        'app_trace_delay': app_trace_delay, 
        'manual_trace_delay': manual_trace_delay, 
        'manual_home_trace_prob': 1.0, 
        'manual_work_trace_prob': 1.0, 
        'manual_othr_trace_prob': 1.0, 
        'met_before_w': 0.79, 
        'met_before_s': 0.9, 
        'met_before_o': 0.9, 
        'max_contacts': max_contacts, 
        'quarantine_length': strategy_quarantine_length, 
        'latent_period': 3, 
        'app_cov': app_coverage, 
        'compliance': compliance, 
        'go_to_school_prob': go_to_school_prob, 
        'wfh_prob': wfh_prob
    }
    
    # Factor
    factor_config = {
        'app_cov': app_coverage, 
        'compliance': compliance, 
        'go_to_school_prob': go_to_school_prob, 
        'wfh_prob': wfh_prob
    }
    
    # Strategy
    strategy_config = {
        'isolate_individual_on_symptoms': True, 
        'isolate_individual_on_positive': True, 
        'isolate_household_on_symptoms': True, 
        'isolate_household_on_positive': True, 
        'isolate_contacts_on_symptoms': False, 
        'isolate_contacts_on_positive': True, 
        'test_contacts_on_positive': False, 
        'do_symptom_testing': True, 
        'do_manual_tracing': True, 
        'do_app_tracing': True, 
        'fractional_infections': True, 
        'testing_delay': testing_delay, 
        'app_trace_delay': app_trace_delay, 
        'manual_trace_delay': manual_trace_delay, 
        'manual_home_trace_prob': 1.0, 
        'manual_work_trace_prob': 1.0, 
        'manual_othr_trace_prob': 1.0, 
        'met_before_w': 0.79, 
        'met_before_s': 0.9, 
        'met_before_o': 0.9, 
        'max_contacts': max_contacts, 
        'quarantine_length': strategy_quarantine_length, 
        'latent_period': 3, 
        'app_cov': app_coverage, 
        'compliance': compliance
    }
    
    return case_config, contacts_config, policy_config, factor_config, strategy_config

In [ ]:
def gp_simulation_result_all(x, y="Effective R"):
    """
    Given a set of input, this function outputs, the simulation result.
    
    :param x: matrix of input to the simulation (from RandomDesign emukit)
    :param y: key output that we want to measure, can be 'Effective R', 'Base R', '# Manual Traces',
              etc (see Utilities.ipynb for more values).
    
    :return: array of simulation results.
    """
    
    # Placeholder to store results
    y = np.zeros((x.shape[0],1))
    rng = np.random.RandomState(42)
    
    # First, generate configurations
    case_config, contacts_config, policy_config, factor_config, strategy_config = get_gp_config()
    
    # Iterate over  all possible configurations
    for i in range(x.shape[0]):
        ## General factors 
        contacts_config['home_sar'] = x[i, 0]
        contacts_config['work_sar'] = x[i, 1]
        contacts_config['other_sar'] = x[i, 2]
        case_config['p_under18'] = x[i, 3]

        ## Policy Factors
        policy_config['quarantine_length'] = x[i, 4]
        strategy_config['quarantine_length'] = x[i, 4]

        policy_config['testing_delay'] = x[i, 5].astype(int)
        strategy_config['testing_delay'] = x[i, 5].astype(int)

        policy_config['app_trace_delay'] = x[i, 6].astype(int)
        strategy_config['app_trace_delay'] = x[i, 6].astype(int)

        policy_config['manual_trace_delay'] = x[i, 7].astype(int)
        strategy_config['manual_trace_delay'] = x[i, 7].astype(int)

        policy_config['go_to_school_prob'] = x[i, 8]
        factor_config['go_to_school_prob'] = x[i, 8]

        policy_config['wfh_prob'] = x[i, 9]
        factor_config['wfh_prob'] = x[i, 9]

        policy_config['max_contacts'] = x[i, 10]
        strategy_config['max_contacts'] = x[i, 10]

        # Compliance Factors
        policy_config['compliance'] = x[i, 11]
        factor_config['compliance'] = x[i, 11]
        strategy_config['compliance'] = x[i, 11]

        policy_config['app_cov'] = x[i, 12]
        factor_config['app_cov'] = x[i, 12]
        strategy_config['app_cov'] = x[i, 12]
        
        # Simulate here, calculate result
        simulate_contacts = EmpiricalContactsSimulator(over18, under18, rng)
        tti_model = TTIFlowModel(rng, **strategy_config)
        n_cases = 10000
        outputs = list()
        ## Perform simulation
        for _ in range(n_cases):
            case = simulate_case(rng, **case_config)
            case_factors = CaseFactors.simulate_from(rng, case, **factor_config)
            contacts = simulate_contacts(case, **contacts_config)
            res = tti_model(case, contacts, case_factors)
            outputs.append(res)
        results = pd.DataFrame(outputs).mean(0)
        y[i, 0] = float(results.loc["Effective R"])
    
    # Return results
    return y

In [ ]:
def gp_simulation_result_general(x, y="Effective R"):
    """
    Given a set of input, this function outputs, the simulation result.
    This functions only consider general variables.
    
    :param x: matrix of input to the simulation (from RandomDesign emukit)
    :param y: key output that we want to measure, can be 'Effective R', 'Base R', '# Manual Traces',
              etc (see Utilities.ipynb for more values).
    
    :return: array of simulation results.
    """
    
    # Placeholder to store results
    y = np.zeros((x.shape[0],1))
    rng = np.random.RandomState(42)
    
    # First, generate configurations
    case_config, contacts_config, policy_config, factor_config, strategy_config = get_gp_config()
    
    # Iterate over  all possible configurations
    for i in range(x.shape[0]):
        ## General factors 
        contacts_config['home_sar'] = x[i, 0]
        contacts_config['work_sar'] = x[i, 1]
        contacts_config['other_sar'] = x[i, 2]
        case_config['p_under18'] = x[i, 3]
        
        # Simulate here, calculate result
        simulate_contacts = EmpiricalContactsSimulator(over18, under18, rng)
        tti_model = TTIFlowModel(rng, **strategy_config)
        n_cases = 10000
        outputs = list()
        for _ in trange(n_cases):
            case = simulate_case(rng, **case_config)
            contacts = simulate_contacts(case, **contacts_config)
            case_factors = CaseFactors.simulate_from(rng, case, **factor_config)
            res = tti_model(case, contacts, case_factors)
            outputs.append(res)
        results = pd.DataFrame(outputs).mean(0)
        y[i, 0] = float(results.loc["Effective R"])
    
    # Return results
    return y

In [ ]:
def gp_simulation_result_policy(x, y="Effective R"):
    """
    Given a set of input, this function outputs, the simulation result.
    This function only consider policy parameters.
    
    :param x: matrix of input to the simulation (from RandomDesign emukit)
    :param y: key output that we want to measure, can be 'Effective R', 'Base R', '# Manual Traces',
              etc (see Utilities.ipynb for more values).
    
    :return: array of simulation results.
    """
    
    # Placeholder to store results
    y = np.zeros((x.shape[0],1))
    rng = np.random.RandomState(42)
    
    # First, generate configurations
    case_config, contacts_config, policy_config, factor_config, strategy_config = get_gp_config()
    
    # Iterate over  all possible configurations
    for i in range(x.shape[0]):
        ## Policy Factors
        policy_config['quarantine_length'] = x[i, 0]
        strategy_config['quarantine_length'] = x[i, 0]

#         policy_config['testing_delay'] = x[i, 1].astype(int)
#         strategy_config['testing_delay'] = x[i, 1].astype(int)

#         policy_config['app_trace_delay'] = x[i, 2].astype(int)
#         strategy_config['app_trace_delay'] = x[i, 2].astype(int)

#         policy_config['manual_trace_delay'] = x[i, 3].astype(int)
#         strategy_config['manual_trace_delay'] = x[i, 3].astype(int)

        policy_config['go_to_school_prob'] = x[i, 1]
        factor_config['go_to_school_prob'] = x[i, 1]

        policy_config['wfh_prob'] = x[i, 2]
        factor_config['wfh_prob'] = x[i, 2]

        policy_config['max_contacts'] = x[i, 3]
        strategy_config['max_contacts'] = x[i, 3]
        
        # Simulate here, calculate result
        simulate_contacts = EmpiricalContactsSimulator(over18, under18, rng)
        tti_model = TTIFlowModel(rng, **strategy_config)
        n_cases = 10000
        outputs = list()
        for _ in trange(n_cases):
            case = simulate_case(rng, **case_config)
            contacts = simulate_contacts(case, **contacts_config)
            case_factors = CaseFactors.simulate_from(rng, case, **factor_config)
            res = tti_model(case, contacts, case_factors)
            outputs.append(res)
        results = pd.DataFrame(outputs).mean(0)
        y[i, 0] = float(results.loc["Effective R"])
    
    # Return results
    return y

In [ ]:
def gp_simulation_result_wfh(x, y="Effective R"):
    """
    Given a set of input, this function outputs, the simulation result.
    This function only consider policy parameters.
    
    :param x: matrix of input to the simulation (from RandomDesign emukit)
    :param y: key output that we want to measure, can be 'Effective R', 'Base R', '# Manual Traces',
              etc (see Utilities.ipynb for more values).
    
    :return: array of simulation results.
    """
    
    # Placeholder to store results
    y = np.zeros((x.shape[0],1))
    rng = np.random.RandomState(42)
    
    # First, generate configurations
    case_config, contacts_config, policy_config, factor_config, strategy_config = get_gp_config()
    
    # Iterate over  all possible configurations
    for i in range(x.shape[0]):
        ## Policy Factors
        policy_config['go_to_school_prob'] = x[i, 0]
        factor_config['go_to_school_prob'] = x[i, 0]

        policy_config['wfh_prob'] = x[i, 1]
        factor_config['wfh_prob'] = x[i, 1]
        
        # Simulate here, calculate result
        simulate_contacts = EmpiricalContactsSimulator(over18, under18, rng)
        tti_model = TTIFlowModel(rng, **strategy_config)
        n_cases = 10000
        outputs = list()
        for _ in trange(n_cases):
            case = simulate_case(rng, **case_config)
            contacts = simulate_contacts(case, **contacts_config)
            case_factors = CaseFactors.simulate_from(rng, case, **factor_config)
            res = tti_model(case, contacts, case_factors)
            outputs.append(res)
        results = pd.DataFrame(outputs).mean(0)
        y[i, 0] = float(results.loc["Effective R"])
    
    # Return results
    return y

In [ ]:
def gp_simulation_result_compliance(x, y="Effective R"):
    """
    Given a set of input, this function outputs, the simulation result.
    Compliance factors only.
    
    :param x: matrix of input to the simulation (from RandomDesign emukit)
    :param y: key output that we want to measure, can be 'Effective R', 'Base R', '# Manual Traces',
              etc (see Utilities.ipynb for more values).
    
    :return: array of simulation results.
    """
    
    # Placeholder to store results
    y = np.zeros((x.shape[0],1))
    rng = np.random.RandomState(42)
    
    # First, generate configurations
    case_config, contacts_config, policy_config, factor_config, strategy_config = get_gp_config()
    
    # Iterate over  all possible configurations
    for i in range(x.shape[0]):
        ## Compliance Factors
        policy_config['compliance'] = x[i, 0]
        factor_config['compliance'] = x[i, 0]
        strategy_config['compliance'] = x[i, 0]

        policy_config['app_cov'] = x[i, 1]
        factor_config['app_cov'] = x[i, 1]
        strategy_config['app_cov'] = x[i, 1]
        
        # Simulate here, calculate result
        simulate_contacts = EmpiricalContactsSimulator(over18, under18, rng)
        tti_model = TTIFlowModel(rng, **strategy_config)
        n_cases = 10000
        outputs = list()
        for _ in trange(n_cases):
            case = simulate_case(rng, **case_config)
            contacts = simulate_contacts(case, **contacts_config)
            case_factors = CaseFactors.simulate_from(rng, case, **factor_config)
            res = tti_model(case, contacts, case_factors)
            outputs.append(res)
        results = pd.DataFrame(outputs).mean(0)
        y[i, 0] = float(results.loc["Effective R"])
    
    # Return results
    return y

### Strategy Optimisation

Contains utilities for optimisation sections.

In [ ]:
def generate_cases(n_cases=10000):
    """
    Used to generate fixed number of cases.
    
    :param n_cases: number of cases and contacts to be generated
    
    :return: list of cases and contacts that are generated.
    """
    
    # GP default config
    rng = np.random.RandomState(42)
    case_config, contacts_config, _, _, _ = get_gp_config()
    simulate_contacts = EmpiricalContactsSimulator(over18, under18, rng)
    list_cases = []
    list_contacts = []
    
    for _ in trange(n_cases):
        case = simulate_case(rng, **case_config)
        contacts = simulate_contacts(case, **contacts_config)
        list_cases.append(case)
        list_contacts.append(contacts)
    
    return list_cases, list_contacts

In [ ]:
def do_simulation_BO(list_cases,
                    list_contacts,
                    factor_config,
                    policy_config,
                    strategy_config):
    """
    Do simulation following given configuration. The function
    is used for Bayesian optimisation experimentation.
    
    :param list_cases: fixed cases list.
    :param list_contacts: fixed contacts list.
    :param factor_config: CaseFactors configuration.
    :param policy_config: policy configuration.
    :param strategy_config: strategy configuration.
    
    :return: simulation output.
    """
    
    outputs = list()
    rng = np.random.RandomState(42)
    tti_model = TTIFlowModel(rng, **strategy_config)
    for i in trange(len(list_cases)):
        # Generate cases and contacts
        case, contacts = list_cases[i], list_contacts[i]
        case_factors = CaseFactors.simulate_from(rng, case, **factor_config)
        res = tti_model(case, contacts, case_factors)
        outputs.append(res)

    return outputs

In [ ]:
def objective_function(x, 
                       variables_to_optimise,
                       list_cases,
                       list_contacts,
                       y_label="Effective R"):
    """
    Objective/ target function to be optimised during 
    Bayesian optimisation procedure.
    
    :param x: matrix of input to the simulation (from RandomDesign emukit)
    :param variables_to_optimise: list of variables to optimise
    :param list_cases: predefined cases list for stability
    :param list_contacts: predefined contacts list for stability
    :param y_label: key output that we want to measure, can be 'Effective R', 'Base R', '# Manual Traces',
              etc (see Utilities.ipynb for more values).
    
    :return: Array of values of y given each x instance.
    """
    
    # Prepare the size
    y = np.zeros((x.shape[0], 1))
    
    # Prepare configuration value
    # Iterate process append result
    for i in range(x.shape[0]):
        bayesian_optimisation_variables = {
            key: value for key, value in zip(variables_to_optimise, x[i,:])
        }
    
        # Fill in default GP config
        case_config, contacts_config, policy_config, factor_config, strategy_config = get_gp_config()
        # Update config value
        for config in [case_config, contacts_config, factor_config, strategy_config, policy_config]:
            for key in config.keys():
                if key in bayesian_optimisation_variables:
                    config[key] = bayesian_optimisation_variables[key]

        # Do simulation, append result
        outputs = do_simulation_BO(list_cases=list_cases, 
                                    list_contacts=list_contacts,
                                    factor_config=factor_config,
                                    policy_config=policy_config,
                                    strategy_config=strategy_config,
                                    )

        results = pd.DataFrame(outputs).mean(0)
        y[i, 0] = float(results.loc[y_label])
    
    return y

In [ ]:
def summarise_search_result(strategy, list_cases, list_contacts):
    """
    Visualise hyperparameter search result in a table.
    
    :param strategy: dictionary with key (variable name), value
        the optimal param.
    :param list_cases: list of cases
    :param list_contacts: list of contacts
    """
    
    case_config, contacts_config, policy_config, factor_config, strategy_config = get_gp_config()
    
    for config in [case_config, contacts_config, factor_config, strategy_config, policy_config]:
        for key in config.keys():
            if key in strategy:
                config[key] = strategy[key]
      
    outputs = do_simulation_BO(
        list_cases=list_cases, 
        list_contacts=list_contacts,
        factor_config=factor_config,
        policy_config=policy_config,
        strategy_config=strategy_config,
        )

    # Display result
    summarise_simulation_results(outputs)

## Appendix: Configurations

This section contains configuration description that we can vary in the experiments. This section is for instructional purposes only. Contains default configuration of config that we used in the experiments, especially for GP-related experiments.

#### Case Config
The following are configurations for each case:
- ```p_under18``` (float): probability of the case being under 18.
- ```infection_proportions``` (dict): probability of being *symp covid neg, symp covid pos, asymp covid pos*.
- ```p_day_noticed_symptoms``` (np.array[float]): distribution of day on which case notices their symptoms. (In our model this is the same as reporting symptoms.)
- ```inf_profile``` (list[float]): describe relative infectiousness of the case for each day of the infectious period. If covid=False, then the value is 0 throughout. We assume 10 days infectiousness following research paper regarding covid. 

In [ ]:
case_config = {
    'p_under18': 0.21, # for sensitivity analysis, not BO (DONE)
    'infection_proportions': { # for sensitivity analysis, not BO (DONE)
        'dist': [0.8333333333333334, 0.1, 0.06666666666666667], 
        'nppl': 120
        }, 
    'p_day_noticed_symptoms': [0, # for sensitivity analysis, not BO (DONE)
                               0.25, 
                               0.25, 
                               0.2, 
                               0.1, 
                               0.05, 
                               0.05, 
                               0.05, 
                               0.05, 
                               0.0], 
    'inf_profile': [0.046966101377360424, # for sensitivity analysis, not BO (DONE) 
                    0.15602255610429985, 
                    0.19829974712514023, 
                    0.18356485224565827, 
                    0.14541407040442172, 
                    0.10500447388376151, 
                    0.07130993362939089, 
                    0.04635772205156416, 
                    0.029167894888682697, 
                    0.017892648289720214]
    }

#### Contact Config

The following are configurations for each contact with a case:
- ```home_sar``` (float): secondary attack rate for household contacts. These contacts are infected at random with attack rates given by the SARs and whether or not the case is symptomatic. If the case is COVID negative, then no contacts are infected.
- ```work_sar``` (float): secondary attack rate for contacts in the work category.
- ```other_sar``` (float): secondary attack rate for contacts in the other category.
- ```asymp_factor``` (float): factor by which to multiply the probability of secondary infection if case is asymptomatic COVID positive (more dangerous).
- ```period``` (int): duration of the simulation (days).

In [ ]:
contacts_config = {
    # for sensitivity analysis, not BO (DONE)
   'home_sar': 0.3, 
   'work_sar': 0.045, 
   'other_sar': 0.045, 
   'period': 10, # not useful
   'asymp_factor': 0.5 # not useful
   }

#### CaseFactors Config

The following are configurations for the factors:
- ```app_cov``` (float): coverage of people using the app.
- ```compliance``` (float): probability of a traced contact isolating correctly.
- ```go_to_school_prob``` (float): fraction of school children attending school.
- ```wfh_prob``` (float): proportion of the population working from home.

In [ ]:
factor_config = {
    # for sensitivity analysis and BO
    'app_cov': 0.35, # non BO (DONE)
    'compliance': 0.8, # non BO (DONE)
    'go_to_school_prob': 0.5, # (DONE)
    'wfh_prob': 0.45 # (DONE)
    }

#### Policy Config

In [ ]:
policy_config = {
    # for BO
    'isolate_individual_on_symptoms':True,  # Isolate the individual after they present with symptoms
    'isolate_individual_on_positive':True,  # Isolate the individual after they test positive
    'isolate_household_on_symptoms':False,  # Isolate the household after individual present with symptoms
    'isolate_household_on_positive':True,  # Isolate the household after individual test positive
    'isolate_contacts_on_symptoms':False,  # Isolate the contacts after individual present with symptoms
    'isolate_contacts_on_positive':True,  # Isolate the contacts after individual test positive
    'test_contacts_on_positive':False,  # Do we test contacts of a positive case immediately, or wait for them to develop symptoms
    'do_symptom_testing':True,  # Test symptomatic individuals
    'do_manual_tracing':True,  # Perform manual tracing of contacts
    'do_app_tracing':True,  # Perform app tracing of contacts
    # not useful
    'fractional_infections':True,  # Include infected but traced individuals as a fraction of their infection period not isolated
    # not useful
    'testing_delay':2,  # Days delay between test and results
    'app_trace_delay':0,  # Delay associated with tracing through the app
    'manual_trace_delay':1,  # Delay associated with tracing manually
    'manual_home_trace_prob':1.0,  # Probability of manually tracing a home contact
    'manual_work_trace_prob':1.0,  # Probability of manually tracing a work contact
    'manual_othr_trace_prob':1.0,  # Probability of manually tracing an other contact
    'met_before_w':1.0,  # Probability of having met a work contact before to be able to manually trace
    'met_before_s':1.0,  # Probability of having met a school contact before to be able to manually trace
    'met_before_o':1.0,  # Probability of having met a other contact before to be able to manually trace
    'max_contacts':23,  # Place a limit on the number of other contacts per day
    # useful for BO
    'quarantine_length':14,  # Length of quarantine imposed on COVID cases (and household)
    # not for BO, maybe for sensitivity analysis
    'latent_period':3,  # Length of a cases incubation period (from infection to start of infectious period)
    # Parameters for CaseFactors simulation
    'app_cov':0.35,
    'compliance':0.8,  # Probability of a traced contact isolating correctly
    'go_to_school_prob':1.0,  # Fraction of school children attending school
    'wfh_prob':0.0,  # Proportion or the population working from home
    }

#### Strategy Config

In [ ]:
strategy_config = {
    'isolate_individual_on_symptoms': True, 
    'isolate_individual_on_positive': True, 
    'isolate_household_on_symptoms': True, 
    'isolate_household_on_positive': True, 
    'isolate_contacts_on_symptoms': False, 
    'isolate_contacts_on_positive': True, 
    'test_contacts_on_positive': False, 
    'do_symptom_testing': True, 
    'do_manual_tracing': True, 
    'do_app_tracing': True, 
    'fractional_infections': True, 
    'testing_delay': 2, 
    'app_trace_delay': 0, 
    'manual_trace_delay': 1, 
    'manual_home_trace_prob': 1.0, 
    'manual_work_trace_prob': 1.0, 
    'manual_othr_trace_prob': 1.0, 
    'met_before_w': 0.79, 
    'met_before_s': 0.9, 
    'met_before_o': 0.9, 
    'max_contacts': 10, 
    'quarantine_length': 14, 
    'latent_period': 3, 
    'app_cov': 0.35, 
    'compliance': 0.8
    }

**Note**: for the case and contact config, we can vary with any values that we want. However, for the policy and strategy config, we select default options from combinations of *{S1, S2, S3, S4, S5}* and *{no_TTI, symptom_based_TTI, test_based_TTI, test_based_TTI_test_contacts}*.

As extension to the default, we can also vary some values on the policy and strategy config.